In [ ]:
pip install -U vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.9/474.9 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.0/355.0 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 103.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB

In [ ]:
from vllm import LLM, SamplingParams

llm = LLM(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    dtype="float16",
    gpu_memory_utilization=0.85,
    enforce_eager = True,
    trust_remote_code=True,
    max_model_len=4096,
)

INFO 12-21 16:07:55 [utils.py:253] non-default args: {'trust_remote_code': True, 'dtype': 'float16', 'max_model_len': 4096, 'gpu_memory_utilization': 0.85, 'disable_log_stats': True, 'enforce_eager': True, 'model': 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

INFO 12-21 16:08:32 [model.py:514] Resolved architecture: Qwen2ForCausalLM
WARNING 12-21 16:08:32 [model.py:2005] Casting torch.bfloat16 to torch.float16.
INFO 12-21 16:08:32 [model.py:1661] Using max model len 4096
INFO 12-21 16:08:36 [scheduler.py:230] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 12-21 16:08:36 [vllm.py:622] Enforce eager set, overriding optimization level to -O0
INFO 12-21 16:08:36 [vllm.py:722] Cudagraph is disabled under eager mode


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

WARNING 12-21 16:08:38 [system_utils.py:136] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 12-21 16:12:56 [llm.py:360] Supported tasks: ['generate']


In [ ]:
sampling_params = SamplingParams(
    n=3,
    temperature=0.6,
    top_p=0.95,
    max_tokens=4096,
)

In [ ]:
import json

records = []
with open("/content/FinalDataset.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        records.append(json.loads(line))

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
from itertools import islice
from tqdm.auto import tqdm

In [ ]:
out_f = open("cots1.jsonl", "a", encoding="utf-8")
base = "https://huggingface.co/datasets/uzaymacar/math-rollouts/resolve/main"
# root = f"{base}/deepseek-r1-distill-qwen-14b/temperature_0.6_top_p_0.95/incorrect_base_solution"
root = f"{base}/deepseek-r1-distill-qwen-14b/temperature_0.6_top_p_0.95/correct_base_solution"
dataset = load_dataset(
    "json",
    data_files=f"{root}/problem_*/problem.json",
    streaming=True
)

start_idx = 0
num_examples = 20


example_iterator = islice(dataset["train"], start_idx, start_idx + num_examples)
for i, ex in enumerate(
    tqdm(example_iterator, total=num_examples, desc="Generating Answers"),
    start=start_idx
):
# for i, ex in enumerate(
#     islice(dataset["train"], start_idx, start_idx + num_examples),
#     start=start_idx
# ):
    problem = ex["problem"]
    prompt = ["<|User|>" + problem+ "<|Assistant|><think>"]
    outputs = llm.generate(prompt, sampling_params)
    answer_text = outputs[0].outputs[0].text

    record = {
        "q_id": i,
        "problem": problem,
        "answer": answer_text
    }

    out_f.write(json.dumps(record, ensure_ascii=False) + "\n")
    out_f.flush()

out_f.close()

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Generating Answers:   0%|          | 0/20 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/3 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

...

Processed prompts:   0%|          | 0/3 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/3 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/3 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]